In [2]:
!pip install pandas
import pandas as pd
import tensorflow
import numpy as np 
from keras.models import Sequential
from keras import optimizers
from keras.layers import Dense, Conv2D, Dropout, Flatten, MaxPooling2D
from keras.applications import MobileNetV2

In [3]:
base_model=MobileNetV2(include_top=False, weights="imagenet", input_shape=(224,224,3))

9406464/9406464 [==============================] - 1s 0us/step


In [4]:
model = Sequential()
model.add(base_model)
model.add(Conv2D(64, (3, 3), activation = 'relu'))
model.add(MaxPooling2D(pool_size = (2, 2)))
model.add(Dropout(0.40))
model.add(Flatten())
model.add(Dense(128,activation='relu'))
model.add(Dropout(0.4))
model.add(Dense(7, activation='sigmoid'))

In [6]:
data=pd.read_csv("C:/Users/karan/Downloads/archive (1)/HAM10000_metadata.csv")
data['image_full_name']=data['image_id']+'.jpg'
X=data[['image_full_name','dx','lesion_id']]

In [7]:
X.sample(5)

,image_full_name,dx,lesion_id
3729,ISIC_0029300.jpg,nv,HAM_0003914
5802,ISIC_0025880.jpg,nv,HAM_0005744
529,ISIC_0024893.jpg,bkl,HAM_0002058
7449,ISIC_0033825.jpg,nv,HAM_0007552
7014,ISIC_0026633.jpg,nv,HAM_0007548


In [9]:
!pip install scikit-learn
from sklearn.model_selection import train_test_split
Y=X.pop('dx').to_frame()
X_train, X_test, y_train, y_test   = train_test_split(X,Y, test_size=0.17, random_state=42)
X_train,X_val,y_train,y_val        =train_test_split(X_train, y_train, test_size=0.17, random_state=42)

In [10]:
train=pd.concat([X_train,y_train],axis=1)
val=pd.concat([X_val,y_val],axis=1)
test=pd.concat([X_test,y_test],axis=1)

In [11]:
from sklearn.preprocessing import LabelEncoder
encoder= LabelEncoder()
encoder.fit(val['dx'])
name_as_indexes_train=encoder.transform(val['dx']) 
val['label']=name_as_indexes_train

In [12]:
encoder=LabelEncoder()
encoder.fit(test['dx'])
name_as_indexes_test=encoder.transform(test['dx']) 
test['label']=name_as_indexes_test

In [13]:
from keras_preprocessing.image import ImageDataGenerator
train_generator = ImageDataGenerator(rescale = 1./255,
                                     rotation_range=10,  
                                     zoom_range = 0.1, 
                                     width_shift_range=0.0,  height_shift_range=0.00) 

In [15]:
train_data= train_generator.flow_from_dataframe(dataframe=train,x_col="image_full_name",y_col="dx",
                                                batch_size=32,directory="C:/Users/karan/Downloads/archive (1)/HAM1000_images",
                                                shuffle=True,class_mode="categorical",target_size=(224,224))

Found 6898 validated image filenames belonging to 7 classes.


In [16]:
test_generator=ImageDataGenerator(rescale = 1./255,rotation_range=10,  
                                     zoom_range = 0.1, 
                                     width_shift_range=0.0,  height_shift_range=0.02)

In [18]:
test_data= test_generator.flow_from_dataframe(dataframe=test,x_col="image_full_name",y_col="dx",
                                              directory="C:/Users/karan/Downloads/archive (1)/HAM1000_images",
                                              shuffle=False,batch_size=1,class_mode=None,target_size=(224,224))

Found 1703 validated image filenames.


In [19]:
val_data=test_generator.flow_from_dataframe(dataframe=val,x_col="image_full_name",y_col="dx",
                                            directory="C:/Users/karan/Downloads/archive (1)/HAM1000_images",
                                            batch_size=64,shuffle=False,class_mode="categorical",target_size=(224,224))

Found 1414 validated image filenames belonging to 7 classes.


In [21]:
from keras.callbacks import ReduceLROnPlateau
learning_control = ReduceLROnPlateau(monitor='val_acc', patience=3, verbose=1, factor=.5, min_lr=0.0001)

In [23]:
from keras.optimizers import Adam,SGD
sgd = optimizers.SGD(learning_rate = 0.01, clipvalue = 0.5)
model.compile(optimizer=sgd,loss="categorical_crossentropy",metrics=["accuracy"])

In [ ]:
history = model.fit_generator(generator=train_data,
                            steps_per_epoch=train_data.samples//train_data.batch_size,
                            validation_data=val_data,
                            verbose=1,
                            validation_steps=val_data.samples//val_data.batch_size,
                            epochs=30,callbacks=[learning_control])

C:\Users\karan\AppData\Local\Temp\ipykernel_14780\1443149548.py:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  history = model.fit_generator(generator=train_data,


Epoch 1/30
215/215 [==============================] - 899s 4s/step - loss: 0.6794 - accuracy: 0.7610 - val_loss: 2.6441 - val_accuracy: 0.6733 - lr: 0.0100
Epoch 2/30
215/215 [==============================] - 910s 4s/step - loss: 0.6078 - accuracy: 0.7820 - val_loss: 4.1304 - val_accuracy: 0.6733 - lr: 0.0100
Epoch 3/30
215/215 [==============================] - 906s 4s/step - loss: 0.5569 - accuracy: 0.8025 - val_loss: 4.1168 - val_accuracy: 0.6726 - lr: 0.0100
Epoch 4/30
215/215 [==============================] - 900s 4s/step - loss: 0.4985 - accuracy: 0.8217 - val_loss: 3.1883 - val_accuracy: 0.6761 - lr: 0.0100
Epoch 5/30
166/215 [======================>.......] - ETA: 3:15 - loss: 0.4560 - accuracy: 0.8364